In [5]:
import json
import matplotlib.pyplot as plt
import numpy as np
from skimage.draw import polygon
import skimage.io
from pathlib import Path
from loguru import logger


class AnnotationTOmask:
    def __init__(self, annotation_file):
        # load dataset
        print('loading annotations ...')
        dataset = json.load(open(annotation_file, 'r'))
        print('annotations loaded!')

        # creating index
        print('creating index...')
        imgToAnns = {ann['image_id']: [] for ann in dataset['annotations']}
        anns =      {ann['id']:       [] for ann in dataset['annotations']}
        for ann in dataset['annotations']:
            imgToAnns[ann['image_id']] += [ann]
            anns[ann['id']] = ann

        imgs      = {im['id']: {} for im in dataset['images']}
        for img in dataset['images']:
            imgs[img['id']] = img

        cats = []
        catToImgs = []
        cats = {cat['id']: [] for cat in dataset['categories']}
        for cat in dataset['categories']:
            cats[cat['id']] = cat
        catToImgs = {cat['id']: [] for cat in dataset['categories']}
        for ann in dataset['annotations']:
            catToImgs[ann['category_id']] += [ann['image_id']]

        print('index created!')

        # create class members
        self.anns = anns
        self.imgToAnns = imgToAnns
        self.catToImgs = catToImgs
        self.imgs = imgs
        self.cats = cats
        self.dataset = dataset


    def getAnnIds(self, imgIds, catIds):
        """
        Get ann ids for given cats from all images
        """
        imgIds = imgIds if type(imgIds) == list else [imgIds]
        catIds = catIds if type(catIds) == list else [catIds]

        if not len(imgIds) == 0:
            anns = sum([self.imgToAnns[imgId] for imgId in imgIds if imgId in self.imgToAnns],[])
        else:
            anns = self.dataset['annotations']
        anns = anns if len(catIds)  == 0 else [ann for ann in anns if ann['category_id'] in catIds]
        ids = [ann['id'] for ann in anns]

        return ids

    def getCatIds(self, catNms=[], catIds=[]):
        """ 
        get integer array of cat ids for given cat names, given cat ids
        """
        if len(catNms) == len(catIds) == 0:
            cats = self.dataset['categories']
        else:
            print('+++++++++++++++++++++')
            cats = self.dataset['categories']
            cats = cats if len(catNms) == 0 else [cat for cat in cats if cat['name']          in catNms]
            cats = cats if len(catIds) == 0 else [cat for cat in cats if cat['id']            in catIds]
        ids = [cat['id'] for cat in cats]
        return ids

    def getImgIds(self, imgIds=[], catIds=[]):
        '''
        return an integer array of img ids for given ids with all given cats
        '''
        ids = set(imgIds)
        for catId in catIds:
            if len(ids) == 0:
                ids = set(self.catToImgs[catId])
            else:
                ids &= set(self.catToImgs[catId])
        return list(ids)

    def loadAnns(self, ids=[]):
        """
        loaded ann objects for integer ids specifying annotations
        """
        if type(ids) == list:
            return [self.anns[id] for id in ids]
        elif type(ids) == int:
            return [self.anns[ids]]

    def loadImgs(self, ids):
        """
        Load loaded img objects with the specified ids.
        """
        return [self.imgs[ids]]

    def getSeg(self, anns):
        """
        get annotations segmentatins
        """
        if len(anns) == 0:
            print('no annotations found')
            return 0

        S = []
        for ann in anns:
            for seg in ann['segmentation']:
                S.append(seg)
        return S

    def segToMask(self, S, h, w ):
         """
         Convert polygon segmentation to binary mask.
           S: polygon segmentation mask, h: target mask height, w: target mask width
         """
         M = np.zeros((h,w))
         for s in S:
             N = len(s)
             rr, cc = polygon(np.array(s[1:N:2]), np.array(s[0:N:2])) # (y, x)
             M[rr, cc] = 1
         return M


def coco_to_mask(coco_filename, output_dir, organ, voxelsize_mm=None, output_type="JPG", show=False, name_prefix="MaskOfPig_"):
    """
    :param coco_filename: coco_filename must include not only name of Coco file, but also it`s full direction (location) in your PC
    :param output_dir: is used for controll output direction (location) in your PC
    :param organ: name of segmentation part rom what we want have mask ""
    :param voxelsize_mm:
    :param output_type: type of "Save fail" of our program| results]
    :param show:
    :return:
    """
    #file_path = 'task_cell track 20200226-dii-30las-2pre1-2020_10_26_13_28_36-coco 1.0/annotations/instances_default.json'

    file_path = coco_filename
    import datetime
    t0 = datetime.datetime.now()
    cv_an = AnnotationTOmask(file_path)
    t1 = datetime.datetime.now()
    cv_an1=cv_an.getImgIds()
    t2 = datetime.datetime.now()
    logger.debug(f"{t1 - t0}, {t2 - t1}")
    logger.debug('getImgIds', cv_an1)
    # extract the region we want to mask ( Right Kidny,  Liver)

    #catIds = cv_an.getCatIds(catNms=['cell'])
    catIds = cv_an.getCatIds(catNms=[organ])
    #catIds = cv_an.getCatIds(catNms=['Liver'])
    #catIds = cv_an.getCatIds(catNms=['Left Kidney'])

    logger.debug('catIds', catIds)
    imgIds = cv_an.getImgIds(catIds=catIds)
    if len(imgIds) == 0:
        logger.warning(f"Label '{organ}' not found.")
    logger.debug(f'imgIds={imgIds}')
    Path(output_dir).mkdir(parents=True, exist_ok=True)

    #TODO allow storing into one file
    #  imgName = cv_an.dataset['images'][0]
    #  M = np.zeros([imgName['width'], imgName['height'],len(cv_an["images"])])
    #  ...

    # create empty files
    for imgName in cv_an.dataset['images']:
        logger.debug(f'imgName={imgName}')
        M = np.zeros([imgName['width'], imgName['height']])
        image_path = Path(output_dir) / (name_prefix+Path(imgName['file_name']).name+'.'+output_type)
        logger.debug(f"image_path={image_path}")
        skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
        # plt.imsave(image_path, np.uint8(M), cmap = 'gray')

    # rewrite images with label
    for im in imgIds:
        logger.debug(f"iM{im}")
        imgName = cv_an.loadImgs(im)[0]
        anns_ids = cv_an.getAnnIds(imgIds=imgName['id'], catIds=catIds)
        anns = cv_an.loadAnns(anns_ids)
        S = cv_an.getSeg(anns)
        M = cv_an.segToMask(S, imgName['width'], imgName['height']) * 255
        if show:
            plt.figure()
            plt.imshow(M)
            plt.show()
            plt.close()
        # image_path = Path(output_dir) / ('MaskOfPIG_'+str(imgName['file_name'])+'.'+output_type)
        image_path = Path(output_dir) / (name_prefix+Path(imgName['file_name']).name+'.'+output_type)
        logger.debug(image_path)
        un = np.unique(M)
        if len(un) < 2:
            logger.warning(f"No data found in annotation {imgName['file_name']}")
        skimage.io.imsave(image_path, np.uint8(M), check_contrast=False)
        plt.imsave(image_path, np.uint8(M), cmap='gray')

    logger.debug('Done')

In [ ]:
Coco_Path="D:/Experimenty/Masky/Ucitel/Validace.json"
Output_Path="D:/Experimenty/Masky/Validace/Liver"
organ="Liver"
coco_to_mask1=coco_to_mask(Coco_Path, Output_Path, organ, voxelsize_mm=None, output_type="PNG", show=False, name_prefix="MaskOfPig_")

2023-05-18 01:16:17.996 | DEBUG    | __main__:coco_to_mask:152 - 0:00:00.015523, 0:00:00
2023-05-18 01:16:17.997 | DEBUG    | __main__:coco_to_mask:153 - getImgIds
2023-05-18 01:16:17.998 | DEBUG    | __main__:coco_to_mask:161 - catIds
2023-05-18 01:16:17.998 | DEBUG    | __main__:coco_to_mask:165 - imgIds=[407, 408, 409, 410, 411, 505, 508, 506, 507, 503, 12660, 12661, 12662, 12663, 12664, 12665, 12666, 12667, 12668, 12669, 12670, 12671, 12672, 12673, 12674, 12675, 12676, 12677, 12678, 12679, 12680, 12681, 12682, 12683, 12684, 12685, 12686, 12687, 12688, 12689, 12690, 12691, 12692, 12693, 12694, 12695, 12696, 12697, 12698, 12699, 12700, 12701, 12702, 12703, 12704, 12705, 12706, 12707, 12708, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462, 463, 464, 465, 466, 467, 468, 469, 470, 471

loading annotations ...
annotations loaded!
creating index...
index created!
+++++++++++++++++++++


2023-05-18 01:16:18.436 | DEBUG    | __main__:coco_to_mask:175 - imgName={'id': 12459, 'width': 512, 'height': 512, 'file_name': 'images/Tx032D_Vensoubor_00869.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
2023-05-18 01:16:18.437 | DEBUG    | __main__:coco_to_mask:178 - image_path=D:\Experimenty\Masky\Validace\Liver\MaskOfPig_Tx032D_Vensoubor_00869.jpg.PNG
2023-05-18 01:16:18.442 | DEBUG    | __main__:coco_to_mask:175 - imgName={'id': 12460, 'width': 512, 'height': 512, 'file_name': 'images/Tx032D_Vensoubor_00868.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captured': 0}
2023-05-18 01:16:18.442 | DEBUG    | __main__:coco_to_mask:178 - image_path=D:\Experimenty\Masky\Validace\Liver\MaskOfPig_Tx032D_Vensoubor_00868.jpg.PNG
2023-05-18 01:16:18.446 | DEBUG    | __main__:coco_to_mask:175 - imgName={'id': 12461, 'width': 512, 'height': 512, 'file_name': 'images/Tx032D_Vensoubor_00867.jpg', 'license': 0, 'flickr_url': '', 'coco_url': '', 'date_captur